In [18]:
pip install numpy pandas

SyntaxError: invalid syntax (2412190004.py, line 1)

In [8]:
pip install datasets

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/536.7 kB ? eta -:--:--
   ------------------------ -------------- 337.9/536.7 kB 10.2 MB/s eta 0:00:01
   ---------------------------------------  532.5/536.7 kB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 536.7/536.7 kB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/166.4 kB ? eta -:--:--
   --------------------------------------- 166.4/166.4 kB 10.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/365.3 kB ? eta -:--:--
   --------------------------------------- 365.3/365.3 kB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/346.2 kB ? eta -:--:--
   ----------

In [15]:
pip install transformers

     ---------------------------------------- 0.0/131.1 kB ? eta -:--:--
     ------------------ -------------------- 61.4/131.1 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 131.1/131.1 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.5 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.5 MB 11.4 MB/s eta 0:00:01
   ----- ---------------------------------- 1.1/8.5 MB 14.6 MB/s eta 0:00:01
   -------- ------------------------------- 1.7/8.5 MB 13.8 MB/s eta 0:00:01
   ----------- ---------------------------- 2.5/8.5 MB 14.9 MB/s eta 0:00:01
   ------------- -------------------------- 2.9/8.5 MB 12.6 MB/s eta 0:00:01
   --------------- ------------------------ 3.2/8.5 MB 11.4 MB/s eta 0:00:01
   ------------------ --------------------- 3.9/8.5 MB 12.3 MB/s eta 0:00:01
   ----

In [10]:
from datasets import load_dataset

# Load the IMDb dataset
dataset = load_dataset("imdb")

# Print dataset info
print(dataset)

# Get a sample of the dataset
sample = dataset['train'][0]
print(sample)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})
{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and

In [20]:
import pandas as pd

In [21]:
# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Check for missing values
print("Missing values:")
print(df.isnull().sum())

Missing values:
text     0
label    0
dtype: int64
